In [1]:
import os
import tensorflow as tf

from dotenv import load_dotenv
from usd_idr_forecasting.models import ModelBuilder
from usd_idr_forecasting.trainers import ColdStartTrainer
from usd_idr_forecasting.configs import ProjectConfig

load_dotenv()
PROJECT_WORKING_DIR = os.getenv('PROJECT_WORKING_DIR')

/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/heykalsayid/.netrc
wandb: Currently logged in as: heykalsayid (danielteam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/heykalsayid/.netrc


In [2]:
yaml_path = os.path.join(PROJECT_WORKING_DIR, 'project_configs.yaml')
project_config = ProjectConfig.from_yaml(yaml_path)
project_config

ProjectConfig(project_name='idrx-forecast', wandb_team_name='danielteam', general_config={'windowing_size': 42, 'target_size': 1, 'batch_size': [8, 16, 32], 'shuffle_buffer_size': None}, models={'learning_rate': 0.0005, 'momentum': 0.9, 'optimizer': ['adam'], 'epochs': 50, 'rnn': {'units': 128, 'return_sequences': True}, 'conv1d': {'filter': 32, 'kernel_size': 3, 'strides': 1, 'padding': 'causal', 'activation': 'relu', 'name': 'conv1d'}, 'max_pooling_1d': 42, 'dropout': 0.2, 'feed_forward_layer': {'dense_1': {'dense': 10, 'activation': 'relu', 'name': 'dense_1'}, 'output': {'dense': 1, 'name': 'output'}}}, dataset_config={'provider': 'yfinance', 'ticker': 'IDR=X', 'start_date': datetime.date(2002, 1, 1), 'end_date': datetime.date(2025, 9, 1), 'test_fraction': 0.2})

In [3]:
builder = ModelBuilder(config=project_config)

In [4]:
builder._build_network(rnn_type='lstm')
model = builder.build()

2026-01-17 17:07:24.538 | SUCCESS  | usd_idr_forecasting.models:_build_network:114 - Successfully Creating Neural Netowkr Model Architecture
2026-01-17 17:07:24.539 | INFO     | usd_idr_forecasting.models:_build_network:115 - Access the model with __class__._model or for compiling with __class__.build


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 42, 1)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ LSTM (LSTM)                     │ (None, 42, 128)        │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 42, 32)         │        12,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ (None, 32)             │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           330 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output (Dense)                  │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 79,349 (309.96 KB)

 Trainable params: 79,285 (309.71 KB)

 Non-trainable params: 64 (256.00 B)

In [5]:
trainer = ColdStartTrainer(config=project_config, model_config=builder.config)
trainer.run(model)

wandb:   37 of 37 files downloaded.  


available preprocessed datasets:
	['train@preprocessed-ds_2025121_172934_626424_batch@8', 'scaler.pkl', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid@preprocessed-ds_2025121_172934_626424_batch@8', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@16', 'valid@preprocessed-ds_2025121_172934_626424_batch@16', 'train@preprocessed-ds_2025121_172934_626424_batch@32', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid@preprocessed-ds_2025121_172934_626424_batch@32', 'train@preprocessed-ds_2025121_172934_626424_batch@16', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@16']


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


['train@preprocessed-ds_2025121_172934_626424_batch@8', 'scaler.pkl', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid@preprocessed-ds_2025121_172934_626424_batch@8', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@8', 'valid_outliers@preprocessed-ds_2025121_172934_626424_batch@16', 'valid@preprocessed-ds_2025121_172934_626424_batch@16', 'train@preprocessed-ds_2025121_172934_626424_batch@32', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@32', 'valid@preprocessed-ds_2025121_172934_626424_batch@32', 'train@preprocessed-ds_2025121_172934_626424_batch@16', 'train_outliers@preprocessed-ds_2025121_172934_626424_batch@16']
train index: 8
valid index: 10


wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


Epoch 1/50
148/148 - 5s - 31ms/step - loss: 0.0097 - mae: 0.0985 - mape: 20037.8027 - mse: 0.0193 - rmse: 0.1391 - val_loss: 0.7498 - val_mae: 1.2494 - val_mape: 148.8637 - val_mse: 1.5678 - val_rmse: 1.2525
Epoch 2/50
148/148 - 2s - 16ms/step - loss: 0.0199 - mae: 0.1443 - mape: 109730.6641 - mse: 0.0398 - rmse: 0.1997 - val_loss: 2.4801 - val_mae: 2.9797 - val_mape: 353.6292 - val_mse: 8.8857 - val_rmse: 2.9813
Epoch 3/50
148/148 - 2s - 16ms/step - loss: 0.0199 - mae: 0.1494 - mape: 101991.1172 - mse: 0.0397 - rmse: 0.1995 - val_loss: 0.0786 - val_mae: 0.3724 - val_mape: 42.3895 - val_mse: 0.1577 - val_rmse: 0.3964
Epoch 4/50
148/148 - 2s - 17ms/step - loss: 0.0224 - mae: 0.1583 - mape: 97646.3281 - mse: 0.0448 - rmse: 0.2119 - val_loss: 0.5240 - val_mae: 1.0235 - val_mape: 121.6163 - val_mse: 1.0496 - val_rmse: 1.0247
Epoch 5/50
148/148 - 2s - 17ms/step - loss: 0.0178 - mae: 0.1358 - mape: 91098.9062 - mse: 0.0355 - rmse: 0.1887 - val_loss: 29.9419 - val_mae: 30.4416 - val_mape: 360

wandb: Adding directory to artifact (/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/models/cold_start_models/lstm)... Done. 0.0s
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


epoch/epoch,▁▂▂▃▃▄▅▅▆▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,▁▆▆█▅▇██▇▇▆██
epoch/mae,▁▆▇█▅▇██▆▆▆▇▇
epoch/mape,▁▇▇▆▆▆▆█▆▇▇▆▇
epoch/mse,▁▆▆█▅▇██▇▇▆██
epoch/rmse,▁▆▆█▅▇██▇▇▇██
epoch/val_loss,▁▂▁▁█▂▁▂▂▆█▆▂
epoch/val_mae,▁▂▁▁█▂▁▂▂▆█▆▂
epoch/val_mape,▁▂▁▁█▂▁▂▂▆█▆▂
+2,...


In [6]:
evaluated_df = trainer.evaluate(split_mode='train')

wandb:   1 of 1 files downloaded.  
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


<Functional name=functional, built=True>


wandb:   37 of 37 files downloaded.  
/Users/heykalsayid/Desktop/myown/machine_learning/usd-idr-forecasting/.venv/lib/python3.11/site-packages/sklearn/base.py:463: InconsistentVersionWarning: Trying to unpickle estimator MinMaxScaler from version 1.2.2 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


split metadata:
test_size: 1190
train_size: 4764
test_fraction: 0.2
test_size_outliers: 1190
train_size_outliers: 4784


wandb:   4 of 4 files downloaded.  
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


prep_sries: <_LoadDataset element_spec=(TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))>
148/148 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
(4722, 1)


wandb: ERROR The nbformat package was not found. It is required to save notebook history.
